# LSTM Model

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

import os
os.chdir("F:/Defi_IA/data")

In [4]:
# Y values
predictions_inter = pd.read_csv('Dataset/predictions_interpolate.csv')
#predictions = pd.read_csv('Dataset/predictions_time_interpolate.csv')
predictions_no_inter = pd.read_csv('Dataset/predictions.csv')

predictions_inter.sort_values(['station', 'test', 'day_index'], inplace=True)
predictions_no_inter.sort_values(['station', 'test', 'day_index'], inplace=True)

#predictions = pd.concat([predictions_no_inter[~predictions_no_inter.test], predictions_inter[predictions_inter.test]])
predictions = predictions_inter

# Feature mesured and interpolated
features_observed_inter = pd.read_csv('Dataset/features_interpolate.csv')
#df_features_observed = pd.read_csv('Dataset/features_time_interpolate.csv')
features_observed_no_inter = pd.read_csv('Dataset/features.csv')

features_observed_inter.sort_values(['station', 'test', 'day_index', 'hour'], inplace=True)
features_observed_no_inter.sort_values(['station', 'test', 'day_index', 'hour'], inplace=True)

#features_observed = pd.concat([features_observed_no_inter[~features_observed_no_inter.test], features_observed_inter[features_observed_inter.test]])
features_observed = features_observed_inter

# Feature from forecast model(ARPEGE)
features_forecast = pd.read_csv('Dataset/features_forecast_final.csv')[['station', 'test', 'day_index', 'hour'] + ['ws', 'p3031', 'u10', 'v10', 't2m', 'd2m', 'r', 'tp', 'msl']]
print(features_observed.shape)
print(features_forecast.shape)

features = pd.merge(features_observed, features_forecast, on=['station', 'test', 'day_index', 'hour'])
features.sort_values(['station', 'test', 'day_index', 'hour'], inplace=True)
features.reset_index(drop=True, inplace=True)
features.head()

# Remove last NA
features.tp.fillna(0, inplace=True)
features = features[(~features.msl.isna()) | features.test]
print(features.shape)
print(f"NA : {features.isna().any().any()}")

(8525400, 12)
(8525400, 13)
(8330400, 21)
NA : True


In [86]:
# The "test" column is a boolean column which is equal to True for test and False for train.
X_train = df[df["test"] == False].copy()
X_test = df[df["test"]].copy()

X_train = X_train[X_train["y"].notnull()]

# y_train is the target variable to train our model.
y_train = X_train["y"].astype(float).copy()

X_test["id"] = X_test["station"].astype(str) + "_" + X_test["day_index"].astype(str)
X_test = X_test.set_index("id")

columns_to_drop = ["y", "test", "station", "day", "day_index"]

X_train = X_train.drop(columns_to_drop, axis = "columns")
X_test = X_test.drop(columns_to_drop, axis = "columns")

X_train[X_train.columns] = MinMaxScaler().fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = MinMaxScaler().fit_transform(X_test[X_test.columns])

In [7]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

df_train = df[df.test==False].fillna(-1)
features_names = list(set(df_train.columns) - set(["station", "y", "day", "test"]))

df_train[features_names] =  MinMaxScaler().fit_transform(df_train[features_names])
df_train.y =  MinMaxScaler().fit_transform(df_train[['y']])

nb_day_station = df_train.station.value_counts()
df_train = df_train[df_train.station.isin(nb_day_station[nb_day_station == 730].index)]

def preprocess_data(df):
    X_train, y_train = [], []
    for _, group in df.groupby('station'):
        data_gen = TimeseriesGenerator(group[features_names].to_numpy(), group.y.to_numpy(), length=16, sampling_rate=1, batch_size=1)
        X_train += [x[0] for x in data_gen]
        y_train += [x[1] for x in data_gen]
    return np.concatenate(X_train, axis=0), np.concatenate(y_train, axis=0)

X_train, y_train = preprocess_data(df_train[df_train.day <= 600])
X_val, y_val = preprocess_data(df_train[df_train.day > 600])

In [35]:
df_train_full[(df_train_full.station==14066001) & df_train_full.station.isna()]

,station,day,hour,day_index,month,ff,t,td,hu,dd,...,u10,v10,t2m,d2m,r,tp,msl,y,Observation,Forecast


In [165]:
df_train.groupby(['station', 'day_index']).day.count().reset_index()

,station,day_index,day
0,14066001,0,24
1,14066001,1,24
2,14066001,2,24
3,14066001,3,24
4,14066001,4,24
...,...,...,...
700,14066001,725,24
701,14066001,726,24
702,14066001,727,24
703,14066001,728,24


In [233]:
df_train

,station,day,hour,day_index,month,ff,t,td,hu,dd,...,u10,v10,t2m,d2m,r,tp,msl,y,Observation,Forecast
33840,14126001,0,0,0,1,0.0,278.50,278.40,99.0,0.0,...,0.215121,2.996554,278.685536,277.975787,95.174006,0.000097,102333.403783,0.5,0.500000,6.791890
33841,14126001,0,1,0,1,0.0,278.95,278.85,99.0,0.0,...,-0.454356,2.690872,277.938222,277.423994,96.486057,0.000372,102357.270034,0.5,0.500000,6.791890
33842,14126001,0,2,0,1,0.0,278.40,278.30,99.0,0.0,...,-1.002666,2.128353,277.281751,276.910831,97.428032,0.001080,102387.484633,0.5,0.500000,6.791890
33843,14126001,0,3,0,1,0.0,277.75,277.65,99.0,0.0,...,-1.453063,1.825482,276.689704,276.385146,97.878996,0.002093,102398.657702,0.5,0.500000,6.791890
33844,14126001,0,4,0,1,0.0,278.05,277.95,99.0,0.0,...,-1.948550,1.567572,276.247757,275.965274,98.013155,0.003609,102353.387251,0.5,0.500000,6.791890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50755,14126001,729,19,729,12,0.0,286.45,285.10,91.5,0.0,...,6.935373,4.823992,286.015884,284.622945,91.233992,15.048403,100785.561436,5.0,1.506818,3.853825
50756,14126001,729,20,729,12,0.0,286.45,284.95,90.5,0.0,...,6.246307,6.004695,285.969937,284.439897,90.421536,15.048419,100798.095814,5.0,1.506818,3.853825
50757,14126001,729,21,729,12,0.0,286.30,284.60,89.5,0.0,...,7.451725,6.451964,285.781005,283.592115,86.525592,15.049211,100824.285403,5.0,1.506818,3.853825
50758,14126001,729,22,729,12,0.0,286.20,284.20,87.5,0.0,...,5.930985,7.499107,285.289158,282.747780,84.469840,15.050266,100780.720963,5.0,1.506818,3.853825


In [249]:
df_train.head(30)

,station,day,hour,day_index,month,ff,t,td,hu,dd,...,u10,v10,t2m,d2m,r,tp,msl,y,Observation,Forecast
33840,14126001,0,0,0,1,0.0,278.50,278.40,99.0,0.0,...,0.215121,2.996554,278.685536,277.975787,95.174006,0.000097,102333.403783,0.5,0.5,6.791890
33841,14126001,0,1,0,1,0.0,278.95,278.85,99.0,0.0,...,-0.454356,2.690872,277.938222,277.423994,96.486057,0.000372,102357.270034,0.5,0.5,6.791890
33842,14126001,0,2,0,1,0.0,278.40,278.30,99.0,0.0,...,-1.002666,2.128353,277.281751,276.910831,97.428032,0.001080,102387.484633,0.5,0.5,6.791890
33843,14126001,0,3,0,1,0.0,277.75,277.65,99.0,0.0,...,-1.453063,1.825482,276.689704,276.385146,97.878996,0.002093,102398.657702,0.5,0.5,6.791890
33844,14126001,0,4,0,1,0.0,278.05,277.95,99.0,0.0,...,-1.948550,1.567572,276.247757,275.965274,98.013155,0.003609,102353.387251,0.5,0.5,6.791890
33845,14126001,0,5,0,1,0.0,277.40,277.30,99.0,0.0,...,-1.950683,0.999966,275.668976,275.414860,98.207822,0.004854,102303.458551,0.5,0.5,6.791890
33846,14126001,0,6,0,1,0.0,278.10,278.00,99.0,0.0,...,-1.483338,1.221486,275.374719,275.110934,98.140239,0.006093,102288.719508,0.5,0.5,6.791890
33847,14126001,0,7,0,1,0.0,278.35,278.25,99.0,0.0,...,-1.920197,0.593731,275.378867,275.061576,97.764279,0.006878,102245.765714,0.5,0.5,6.791890
33848,14126001,0,8,0,1,0.0,278.55,278.45,99.0,0.0,...,-2.231030,-0.090387,275.471963,274.998049,96.690029,0.007208,102175.807467,0.5,0.5,6.791890
33849,14126001,0,9,0,1,0.0,279.75,279.65,99.0,0.0,...,-3.382155,2.841560,276.396588,275.689266,95.110825,0.007423,102150.835702,0.5,0.5,6.791890


In [250]:
y

,station,day_index,Observation,Forecast,y,error
0,14126001,0,0.500000,6.791890,0.5,-6.291890
1,14126001,1,0.500000,11.050955,3.5,-7.550955
2,14126001,2,3.500000,2.024017,0.0,-2.024017
3,14126001,3,0.000000,0.212313,0.0,-0.212313
4,14126001,4,0.000000,2.331213,1.0,-1.331213
...,...,...,...,...,...,...
700,14126001,725,5.500000,4.551255,5.0,0.448745
701,14126001,726,5.000000,1.245234,6.0,4.754766
702,14126001,727,6.000000,8.121815,3.5,-4.621815
703,14126001,728,3.500000,10.558058,3.5,-7.058058


In [248]:
X[1]

array([[ 0.00000000e+00,  5.00000000e-01,  1.10509550e+01,
         1.00000000e+00,  0.00000000e+00,  2.82350000e+02,
         2.81200000e+02,  9.25000000e+01,  0.00000000e+00,
         0.00000000e+00,  9.18181540e+00,  1.58196921e+02,
        -3.41293763e+00,  8.52317745e+00,  2.81551216e+02,
         2.80018943e+02,  9.00754953e+01, -1.48205525e-04,
         1.00548307e+05,  1.00000000e+00,  5.00000000e-01,
         1.10509550e+01,  1.00000000e+00,  0.00000000e+00],
       [ 2.82600000e+02,  2.81600000e+02,  9.35000000e+01,
         0.00000000e+00,  7.78858062e-19,  9.13313242e+00,
         1.59644758e+02, -3.17486578e+00,  8.56443258e+00,
         2.81231025e+02,  2.79985746e+02,  9.18505857e+01,
        -1.19331834e-04,  1.00477306e+05,  2.00000000e+00,
         5.00000000e-01,  1.10509550e+01,  1.00000000e+00,
         0.00000000e+00,  2.82650000e+02,  2.81550000e+02,
         9.30000000e+01,  0.00000000e+00,  9.70439186e-20],
       [ 9.03139721e+00,  1.66077769e+02, -2.16918932e

In [218]:
X_train

array([[[ 0.00000000e+00,  2.00000000e+00,  2.32727720e+00, ...,
          7.93563450e+01,  6.20124094e-04,  1.01324017e+05],
        [ 1.00000000e+00,  2.00000000e+00,  2.32727720e+00, ...,
          8.07877722e+01,  6.00018094e-04,  1.01309944e+05],
        [ 2.00000000e+00,  2.00000000e+00,  2.32727720e+00, ...,
          8.18155788e+01,  6.16116122e-04,  1.01247589e+05],
        ...,
        [ 2.10000000e+01,  2.00000000e+00,  2.32727720e+00, ...,
          9.43524505e+01,  9.50916245e+00,  1.00929117e+05],
        [ 2.20000000e+01,  2.00000000e+00,  2.32727720e+00, ...,
          9.47182645e+01,  9.56011812e+00,  1.00918283e+05],
        [ 2.30000000e+01,  2.00000000e+00,  2.32727720e+00, ...,
          9.39531143e+01,  9.56676236e+00,  1.00893025e+05]],

       [[ 0.00000000e+00,  1.50681818e+00,  5.87712700e-01, ...,
          8.76719314e+01,  1.10450776e-02,  1.00582181e+05],
        [ 1.00000000e+00,  1.50681818e+00,  5.87712700e-01, ...,
          8.80002146e+01,  2.71523029e

In [229]:
np.array([group[list_features].to_numpy() for _, group in df_train.groupby(['station', 'day_index'])])

array([[[ 0.00000000e+00,  5.00000000e-01,  6.79188970e+00, ...,
          9.51740064e+01,  9.68655834e-05,  1.02333404e+05],
        [ 1.00000000e+00,  5.00000000e-01,  6.79188970e+00, ...,
          9.64860566e+01,  3.72198355e-04,  1.02357270e+05],
        [ 2.00000000e+00,  5.00000000e-01,  6.79188970e+00, ...,
          9.74280315e+01,  1.07953740e-03,  1.02387485e+05],
        ...,
        [ 2.10000000e+01,  5.00000000e-01,  6.79188970e+00, ...,
          8.43674576e+01,  2.11604144e-01,  1.00733856e+05],
        [ 2.20000000e+01,  5.00000000e-01,  6.79188970e+00, ...,
          8.68668079e+01,  2.52173642e-01,  1.00644998e+05],
        [ 2.30000000e+01,  5.00000000e-01,  6.79188970e+00, ...,
          8.97063368e+01,  2.55235890e-01,  1.00583310e+05]],

       [[ 0.00000000e+00,  5.00000000e-01,  1.10509550e+01, ...,
          9.00754953e+01, -1.48205525e-04,  1.00548307e+05],
        [ 1.00000000e+00,  5.00000000e-01,  1.10509550e+01, ...,
          9.18505857e+01, -1.19331834e

In [217]:
for _, group in df_train.groupby(['station', 'day_index']):
    print(_)
    print(group[list_features].to_numpy())
    break

(14126001, 0)
[[ 0.00000000e+00  5.00000000e-01  6.79188970e+00  1.00000000e+00
   0.00000000e+00  2.78500000e+02  2.78400000e+02  9.90000000e+01
   0.00000000e+00  0.00000000e+00  2.99752574e+00  1.84054860e+02
   2.15121178e-01  2.99655353e+00  2.78685536e+02  2.77975787e+02
   9.51740064e+01  9.68655834e-05  1.02333404e+05]
 [ 1.00000000e+00  5.00000000e-01  6.79188970e+00  1.00000000e+00
   0.00000000e+00  2.78950000e+02  2.78850000e+02  9.90000000e+01
   0.00000000e+00  2.23339516e-21  2.72028680e+00  1.70328966e+02
  -4.54355505e-01  2.69087217e+00  2.77938222e+02  2.77423994e+02
   9.64860566e+01  3.72198355e-04  1.02357270e+05]
 [ 2.00000000e+00  5.00000000e-01  6.79188970e+00  1.00000000e+00
   0.00000000e+00  2.78400000e+02  2.78300000e+02  9.90000000e+01
   0.00000000e+00  3.97697073e-17  2.35203281e+00  1.54799117e+02
  -1.00266552e+00  2.12835271e+00  2.77281751e+02  2.76910831e+02
   9.74280315e+01  1.07953740e-03  1.02387485e+05]
 [ 3.00000000e+00  5.00000000e-01  6.7918

In [287]:
from sklearn.model_selection import train_test_split
list_features = ['Observation', 'Forecast', 'month'] + ['ff', 't', 'td', 'hu', 'dd', 'precip'] + ['ws', 'p3031', 'u10', 'v10', 't2m', 'd2m', 'r', 'tp', 'msl']

df_train_full = pd.merge(features[~features.test], predictions[~predictions.test].drop(['day', 'month'], axis=1), on=['station', 'test', 'day_index']).drop('test', axis=1)
#list_station = df_train_full.station.unique()[1:2]

#df_train = df_train_full[df_train_full.station.isin(list_station)].copy()
# 14126001
# 14047002
df_train = df_train_full[df_train_full.station == 14047002].copy()

for col in list_features:
    df_train[col] = df_train[col].fillna(df_train[col].mean())
#df_train.dropna(inplace=True)

#df_train['id'] = df_train["station"].astype(str) + "_" + df_train["day_index"].astype(str)
#df_train = df_train[df_train.groupby('id')['id'].transform('size') == 24]

#Y
df_forecast_train = df_train.groupby(['station', 'day_index']).first()[['Observation', 'Forecast', 'y']].reset_index()
df_forecast_train['error'] = df_forecast_train.y - df_forecast_train.Forecast
min_error_train, max_error_train = df_forecast_train.error.min(), df_forecast_train.error.max()
mean_error_train, std_error_train = df_forecast_train.error.mean(), df_forecast_train.error.std()
min_max_error = 2 * (max_error_train - min_error_train)
df_forecast_train['error'] = (df_forecast_train['error'] - min_error_train) / (max_error_train - min_error_train)
#df_forecast_train['error'] = df_forecast_train['error'] / std_error_train
y = df_forecast_train#.error.to_numpy()

#X
df_train[list_features] = MinMaxScaler().fit_transform(df_train[list_features])

X = np.array([group[list_features].to_numpy() for _, group in df_train.groupby(['station', 'day_index'])])#.reshape(df_train.shape[0]//24, len(list_features), 24)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=4
)

forecast_train, forecast_test = y_train.Forecast.to_numpy(), y_test.Forecast.to_numpy()
y_train, y_test = y_train.error.to_numpy(), y_test.error.to_numpy()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

C:\Users\mathi\anaconda3\envs\DataScience\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\mathi\anaconda3\envs\DataScience\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


(634, 24, 18)
(634,)
(71, 24, 18)
(71,)


In [280]:
min_error_train, max_error_train

(-27.299407999999993, 14.3592844)

In [275]:
y_train.min

array([0.99268793, 0.97804804, 0.99915411, 0.99950757, 0.98556425,
       0.99315233, 0.77223766, 0.9531546 , 0.96804014, 0.96102987,
       0.88753125, 0.59820474, 0.88011859, 0.99921466, 1.        ,
       1.        , 0.99795031, 0.99833257, 0.99992577, 0.83538692,
       0.99841932, 0.99993095, 0.86589195, 0.99924657, 0.99939968,
       1.        , 1.        , 0.99843004, 0.99968698, 0.99371064,
       0.94643665, 0.97589595, 1.        , 0.99826191, 0.91791406,
       0.71217368, 0.99955557, 0.99817422, 1.        , 0.99995559,
       1.        , 0.99722065, 0.9691205 , 0.96869469, 1.        ,
       1.        , 0.96082304, 0.97925274, 0.9148843 , 1.        ,
       0.9994947 , 1.        , 0.83301308, 0.94438277, 0.98942817,
       1.        , 0.99946585, 0.99605219, 1.        , 1.        ,
       1.        , 0.99792143, 0.99813525, 0.99779677, 0.98912047,
       0.96580651, 0.83553984, 0.97905718, 0.99969196, 0.99961243,
       1.        , 1.        , 0.99064907, 0.91530366, 1.     

In [264]:
from sklearn import metrics
# Custom MAPE

def get_reel_y(x):
    #return x# * std_error_train
    #return x * std_error_train + mean_error_train
    return x * (max_error_train - min_error_train) + min_error_train

def MAPE(y_true, y_pred):
    return metrics.mean_absolute_percentage_error(get_reel_y(y_true) + 1, y_pred + 1)

def MAPE_Error(y_true, y_pred, forecast):
    return MAPE(y_true + forecast, y_pred + forecast)

def MAPE_Error_scaled(y_true, y_pred, forecast):
    return MAPE_Error(get_reel_y(y_true), get_reel_y(y_pred), forecast)

def model_results(model):
    print(f"MAPE on train set: {MAPE_Error_scaled(y_train, model.predict(X_train).reshape(X_train.shape[0]), forecast_train)}")
    print(f"MAPE on test set: {MAPE_Error_scaled(y_test, model.predict(X_test).reshape(X_test.shape[0]), forecast_test)}")

In [262]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization

def lstm_model():
    model = Sequential()
    model.add(LSTM(32, input_shape=X_train.shape[1:], activation='relu', return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(LSTM(32, activation='relu'))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='mse', optimizer='adam', metrics='mean_absolute_percentage_error')
    
    return model

lstm_model().summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_126 (LSTM)             (None, 24, 32)            6528      
                                                                 
 lstm_127 (LSTM)             (None, 32)                8320      
                                                                 
 dense_126 (Dense)           (None, 16)                528       
                                                                 
 dense_127 (Dense)           (None, 1)                 17        
                                                                 
Total params: 15,393
Trainable params: 15,393
Non-trainable params: 0
_________________________________________________________________


In [263]:
model = lstm_model()
model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=64)
model_results(model)

Epoch 1/10
9/9 [==============================] - 3s 182ms/step - loss: 0.0199 - mean_absolute_percentage_error: 951211.2500 - val_loss: 0.0108 - val_mean_absolute_percentage_error: 14.3807
Epoch 2/10
9/9 [==============================] - 2s 195ms/step - loss: 0.0094 - mean_absolute_percentage_error: 1103649.3750 - val_loss: 0.0064 - val_mean_absolute_percentage_error: 10.2711
Epoch 3/10
9/9 [==============================] - 2s 176ms/step - loss: 0.0077 - mean_absolute_percentage_error: 1179099.0000 - val_loss: 0.0049 - val_mean_absolute_percentage_error: 8.5156
Epoch 4/10
9/9 [==============================] - 2s 220ms/step - loss: 0.0072 - mean_absolute_percentage_error: 1065782.0000 - val_loss: 0.0047 - val_mean_absolute_percentage_error: 7.9259
Epoch 5/10
9/9 [==============================] - 2s 207ms/step - loss: 0.0070 - mean_absolute_percentage_error: 1110832.6250 - val_loss: 0.0047 - val_mean_absolute_percentage_error: 7.7476
Epoch 6/10
9/9 [==============================] -

In [254]:
model.predict(X_test).reshape(X_test.shape[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [255]:
get_reel_y(model.predict(X_test).reshape(X_test.shape[0]))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [256]:
y_train

array([-1.32727720e+00,  9.25903034e-01, -7.52505540e+00, -4.32314400e-02,
       -2.29966660e+00, -1.03668800e+00, -5.67052827e+00, -9.29119050e-01,
       -3.07947227e+00, -2.86718100e+00, -4.04858400e+00, -1.31832200e+00,
       -2.81977500e+00, -4.20340340e-02,  0.00000000e+00,  0.00000000e+00,
       -1.75690190e-01, -3.30172780e+00,  0.00000000e+00, -5.27033300e+00,
       -5.71009440e-02, -5.15572700e-01, -5.65668500e+00, -2.67636694e-19,
        4.60115350e-01,  0.00000000e+00,  5.00000000e-01,  0.00000000e+00,
        4.98840124e-01, -3.94643430e-01,  6.31047500e-02, -2.26411700e+00,
        0.00000000e+00, -1.82968530e-01, -3.78435700e+00, -6.75412027e+00,
        0.00000000e+00,  3.88297040e-01, -3.23886130e-02, -4.59068500e-02,
        1.44015802e+00, -1.54339850e+00,  5.51626700e-01, -1.39084950e+00,
       -1.46040280e+00,  0.00000000e+00, -1.65440400e+00, -2.49150040e+00,
        4.06080164e-01,  0.00000000e+00, -2.74718700e-01,  5.45822681e+00,
       -1.53549166e+01, -